# Ejemplo(Falta agregar un costo de almacenamiento)
Juan vende cervezas artesanales de Lunes a Viernes. Las compra por 300 pesos y vende por 1.500 pesos cada una.
Juan maneja su inventario de forma semanal, de la siguiente manera:
* El Lunes al comienzo del día Juan hace la contabilidad de costos, y cuenta cuantas unidades tiene en inventario. Según su nivel de inventario, tiene que tomar una decisión.
    
    * Comprar una cantidad i para atender la clientela la semana j, sin sobrepasar su capacidad de S unidades.
    * No hacer ningún pedido esta semana.
* Finalmente el pedido lo hace el lunes en la mañana y llega en ese mismo instante, Juan vende las cervezas durante la semana.

    
* Si un cliente quiere comprar una cerveza pero no hay disponible, Juan consideraque pierde 1200 pesos por cada cerveza no vendida por falta de inventario. (Precio de Venta - Costo de compra)
    
* Juan vende entre 0 y 400 cervezas por semana. (De momento demanda Uniforme)
    

* El frigobar de juan tiene una capacidad de 500 cervezas (S=500)
* Si la demanda durante la semana sobrepasa el inventario se considera una perdida del beneficio que podría haber recibido ((Precio venta - Precio Compra) * Cantidad de cervezas fuera de inventario)
* Juan pide dinero prestado, por lo cual considera una tasa de descuento del dinero de un 10% semanal

## ¿Ayude a decidir a Juan cuando comprar, según el inventario que tenga?

In [2]:
import numpy as np
from numpy import random
import collections
import time

In [154]:
# Información problema
S=1000#00#20#0 #capacidad max y tamaño de lote
a=0#00#0#3#0 #Demanda minima
b=500#15#0 #Demanda maxima
cost=300#precio cerveza y costo perdida clientes
price=1500
tasa=1


#### Generó las demandas

In [155]:
random.seed(10)
total=50000
x=random.randint(a,b+1, size=(total))
Dem_frec = collections.Counter(x)

### Comiezo a generar mi matriz de probaibilidades
Esta matriz se estructura de la siguiente forma

    Prob[i][j]: es la probailidad de que el inventario i pase al nivel de inventario j en el siguente periodo


In [156]:
prob =np.zeros([S+1,S+1])
for i in range(S+1):
    for j in range(S+1):
        dem=i-j
        if(S>=dem>=0):
            if(dem<i):
               prob[i][j] =Dem_frec[dem]/total
    prob[i][0] = 1 - sum(prob[i][0:])



[250.6954399999998,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.0000000000000004,
 1.00000000000



Tomando en cuenta que realizar la compra en el periodo t, donde el nivel de inventarios es de i, no afectaria al expected reward del mismo periodo t, ya que la compra afecta al nivel de inventarios del perido siguente. Es por esto que la desición no afecta al expected reward del mismo periodo en donde se toma.

Es por ello que para almacenar el expected reward, se utilizará al siguiente estructura:
    
    reward[i]:= el expected reward ante un nivel de inventario i.


In [160]:
# aquí se ajustan los costos de
# D>i o D<i

reward=np.zeros(S+1)
for i in range(S+1):
    r=0
    for dem in range(a,b+1):
        p=Dem_frec[dem]/total
        r_dem=price*min(dem,i)
        if(dem>=i):
            r_dem+=-(price-cost)*(dem-i)
        else:
            #if(dem<i):#Agrego costo de inventario sobrante
            r_dem+= -(cost)*(i-dem)
        r+= p*(r_dem)
    #print(r)
    reward[i]=r

# Comenzamos a estructurar esto

## Value Determination Equations

In [131]:
#funcion para obtener el valor de la ecuaciones
def Value_Determination_Equations(decisions,prob,descuento,costo,revenues,S):
    L=len(decisions)
    aux=np.zeros([L,L])
    B=np.zeros(L)
    #print(B)
    for i in range(L):
        cant_comprada_i=decisions[i]
        if cant_comprada_i>0:
        #Juan decide comprar mas cerveza.
            if(cant_comprada_i+i<=S):
            # La compra más el inventario debe ser menor a Cap max
                B[i]=revenues[cant_comprada_i+i]
                B[i]+=-costo*(cant_comprada_i)
                aux[i]=prob[cant_comprada_i+i]
            else:
                B[i]+=-costo*(cant_comprada_i)*10000
                aux[i]=prob[i]
        else:
            aux[i]=prob[i]
            B[i]=revenues[i]
    aux = np.insert(aux, aux.shape[1], np.zeros(L), 1)
    #aux = np.insert(aux, aux.shape[0], np.zeros(L+1), 0)
    a = np.identity(L)
    a = np.insert(a, a.shape[1], np.ones(L), 1)
    #print("a->",a)
    #print("aux->",aux)
    A = a-(descuento*aux)
    A = np.insert(A, A.shape[0], np.zeros(L+1), 0)
    A[L,L-1] = 1
    #print("A->",A)
    B = np.append(B, 0)
    #print("B->",B)
    X = np.linalg.solve(A,B)
    #print("X->",X)
    g = X[L]
    X = X[:-1]
    #print("X sin g ->",X)
    return [X,g]
    

# Howard’s Policy Iteration Method

In [143]:
def T_delta(prob,values,descuento,costo,revenues,g):
    L=len(values)
    T=np.zeros(L)
    Cant=np.zeros(L)
    for i in range(L):
        lista=[]
        lista_2=[]
        for j in range(i,L):
            lista.append(revenues[j]-g-costo*(j-i)+(descuento*sum(prob[j]*values)))
            lista_2.append(j-i)
        T[i]=max(lista)#Con i=S soy indiferente a comprar o no
        indice=lista.index(T[i])
        Cant[i]=lista_2[indice]
    return [T,Cant]

In [152]:
def Search_OP(decisions,prob,descuento,costo,revenues,S):
   
    #(D,m_prob,descuento,c,states_reveneus)
    V_D_E, G = Value_Determination_Equations(decisions,prob,descuento,costo,revenues,S)
    Los_T, D_T = T_delta(prob,V_D_E,descuento,costo,revenues,G)
    val_Op=np.around(Los_T,4)==np.around(V_D_E,4)
    D_p = np.copy(decisions)
    if(sum(val_Op)==len(val_Op)):
        return(decisions)
    else:
        for i in range(len(val_Op)):
            if (val_Op[i]!=True):
                decisions[i]=D_T[i].astype('int')
        return(Search_OP(decisions,prob,descuento,costo,revenues,S))

In [163]:
D=np.ones(S+1).astype('int')
start_time = time.time()
Optimo=Search_OP(D,prob,tasa,cost,reward,S)
print("--- %s Segundos ---" % (time.time() - start_time))
Optimo

--- 356.964124917984 Segundos ---


array([445, 444, 443, ...,   0,   0,   0])

5.933333333333334